# Binary Options Pricing
By Padraic McAtee
    
My notation here is based on the original Cox, Ross and Rubinstein paper.

In [3]:
import numpy as np
import math as m
import timeit

def bop(n,t,S,v):
    dt = t/n
    u = m.exp(v*m.sqrt(dt))
    d = 1/u
    Pm = np.zeros((n+1, n+1))
    for j in range(n+1):
        for i in range(j+1):
            Pm[i,j] = S*m.pow(d,i) * m.pow(u,j-i)
    return Pm

I generated the pricing tree for a few n values...

In [4]:
n = 5
t = 200/365
S = 100
v = .3
x = bop(n,t,S,v)
n = 17
z = bop(n,t,S,v)

print('n = 5:\n',np.matrix(x.astype(int)))
print('n = 17:\n',np.matrix(z.astype(int)))

n = 5:
 [[100 110 121 134 148 164]
 [  0  90 100 110 121 134]
 [  0   0  81  90 100 110]
 [  0   0   0  74  81  90]
 [  0   0   0   0  67  74]
 [  0   0   0   0   0  60]]
n = 17:
 [[100 105 111 117 124 130 138 145 153 162 171 180 190 201 212 224 236 249]
 [  0  94 100 105 111 117 124 130 138 145 153 162 171 180 190 201 212 224]
 [  0   0  89  94 100 105 111 117 124 130 138 145 153 162 171 180 190 201]
 [  0   0   0  85  89  94  99 105 111 117 124 130 138 145 153 162 171 180]
 [  0   0   0   0  80  85  89  94 100 105 111 117 124 130 138 145 153 162]
 [  0   0   0   0   0  76  80  85  89  94 100 105 111 117 124 130 138 145]
 [  0   0   0   0   0   0  72  76  80  85  89  94  99 105 111 117 124 130]
 [  0   0   0   0   0   0   0  68  72  76  80  85  89  94  99 105 111 117]
 [  0   0   0   0   0   0   0   0  64  68  72  76  80  85  89  94 100 105]
 [  0   0   0   0   0   0   0   0   0  61  64  68  72  76  80  85  89  94]
 [  0   0   0   0   0   0   0   0   0   0  58  61  64  68  72  76  80 

After noticing the recursive pattern in the tree, I generated the set of all unique numbers in the matrix as an ordered 1d array and looped through the elements of the pricing matrix calling values from the unique set. 

In [5]:
def better_bop(n,t,S,v):
    dt = t/n
    u = m.exp(v*m.sqrt(dt))
    d = 1/u
    ups = np.zeros(n+1)
    dwns = np.zeros(n+1)
    tot = np.zeros(2*n+1)
    Pm = np.zeros((n+1, n+1))
    tmp = np.zeros((2,n+1))
    for j in range(n+1):
        tmp[0,j] = S*m.pow(d,j)
        tmp[1,j] = S*m.pow(u,j)
    tot = np.unique(tmp)
    c = n
    for i in range(c+1):
        for j in range(c+1):
            Pm[i,j-c-1] = tot[(n-i)+j]
        c=c-1
    return Pm
trial = better_bop(n,t,S,v)
print('n = 17:\n',np.matrix(trial.astype(int)))

n = 17:
 [[100 105 111 117 124 130 138 145 153 162 171 180 190 201 212 224 236 249]
 [  0  94 100 105 111 117 124 130 138 145 153 162 171 180 190 201 212 224]
 [  0   0  89  94 100 105 111 117 124 130 138 145 153 162 171 180 190 201]
 [  0   0   0  85  89  94 100 105 111 117 124 130 138 145 153 162 171 180]
 [  0   0   0   0  80  85  89  94 100 105 111 117 124 130 138 145 153 162]
 [  0   0   0   0   0  76  80  85  89  94 100 105 111 117 124 130 138 145]
 [  0   0   0   0   0   0  72  76  80  85  89  94 100 105 111 117 124 130]
 [  0   0   0   0   0   0   0  68  72  76  80  85  89  94 100 105 111 117]
 [  0   0   0   0   0   0   0   0  64  68  72  76  80  85  89  94 100 105]
 [  0   0   0   0   0   0   0   0   0  61  64  68  72  76  80  85  89  94]
 [  0   0   0   0   0   0   0   0   0   0  58  61  64  68  72  76  80  85]
 [  0   0   0   0   0   0   0   0   0   0   0  55  58  61  64  68  72  76]
 [  0   0   0   0   0   0   0   0   0   0   0   0  52  55  58  61  64  68]
 [  0   0   0   

Testing for consistency and timing...

In [6]:
%%timeit
method1 = bop(n,t,S,v)

10000 loops, best of 3: 163 µs per loop


In [7]:
%%timeit
method2 = better_bop(n,t,S,v)

10000 loops, best of 3: 95.6 µs per loop


In [8]:
method1 = bop(n,t,S,v)
method2 = better_bop(n,t,S,v)
print('\nConsistent entries?: ' , np.allclose(method1,method2)) #tests if the matrices are equal


Consistent entries?:  True


Method 2 performs much quicker giving the same results.

## Working Backwards to find the *value* of the option
From here, I determined the value of the option based on stike price and value at earlier nodes (as shown in the paper) was very simple to implement in this matrix. 

In [9]:
def OptionsVal(n, S, K, r, v, T, PC):
    dt = T/n                    
    u = m.exp(v*m.sqrt(dt)) 
    d = 1/u                     
    p = (m.exp(r*dt)-d)/(u-d)   
    Pm = np.zeros((n+1, n+1))   
    Cm = np.zeros((n+1, n+1))
    tmp = np.zeros((2,n+1))
    for j in range(n+1):
        tmp[0,j] = S*m.pow(d,j)
        tmp[1,j] = S*m.pow(u,j)
    tot = np.unique(tmp)
    c = n
    for i in range(c+1):
        for j in range(c+1):
            Pm[i,j-c-1] = tot[(n-i)+j]
        c=c-1
    for j in range(n+1, 0, -1):
        for i in range(j):
            if (PC == 1):                               
                if(j == n+1):
                    Cm[i,j-1] = max(K-Pm[i,j-1], 0)     
                else:
                    Cm[i,j-1] = m.exp(-.05*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]) 
            if (PC == 0):                               
                if (j == n + 1):
                    Cm[i,j-1] = max(Pm[i,j-1]-K, 0)     
                else:
                    Cm[i,j-1] = m.exp(-.05*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j])  
    return [Pm,Cm] 

In [10]:
S = 100
k = 100
r = .05
v = .3
T = 20/36
n = 17
PC = 0
Pm,CmC = OptionsVal(n,S,k,r,v,T,PC)
PC = 1
_,CmP= OptionsVal(n, S, k, r, v, T, PC)
print('Pricing:\n',np.matrix(Pm.astype(int)))
print('Call Option:\n',np.matrix(CmC.astype(int)))
print('Put Option:\n',np.matrix(CmP.astype(int)))

Pricing:
 [[100 105 111 117 124 131 138 146 154 162 172 181 191 202 213 225 238 251]
 [  0  94 100 105 111 117 124 131 138 146 154 162 172 181 191 202 213 225]
 [  0   0  89  94 100 105 111 117 124 131 138 146 154 162 172 181 191 202]
 [  0   0   0  84  89  94 100 105 111 117 124 131 138 146 154 162 172 181]
 [  0   0   0   0  80  84  89  94 100 105 111 117 124 131 138 146 154 162]
 [  0   0   0   0   0  76  80  84  89  94 100 105 111 117 124 131 138 146]
 [  0   0   0   0   0   0  72  76  80  84  89  94 100 105 111 117 124 131]
 [  0   0   0   0   0   0   0  68  72  76  80  84  89  94 100 105 111 117]
 [  0   0   0   0   0   0   0   0  64  68  72  76  80  84  89  94 100 105]
 [  0   0   0   0   0   0   0   0   0  61  64  68  72  76  80  84  89  94]
 [  0   0   0   0   0   0   0   0   0   0  58  61  64  68  72  76  80  84]
 [  0   0   0   0   0   0   0   0   0   0   0  55  58  61  64  68  72  76]
 [  0   0   0   0   0   0   0   0   0   0   0   0  52  55  58  61  64  68]
 [  0   0   0  